# 🔍 06번: Hybrid Search + Re-ranking

## 🎯 학습 목표
- **Vector 검색의 한계** 이해 (조문 번호, 법률 용어)
- **BM25 키워드 검색**으로 정확한 조문 찾기
- **Hybrid Search** (Vector + BM25)
- **Re-ranking**으로 최종 정확도 극대화! ⭐

## 💡 핵심 아이디어

```
질문: "도박 빚은 갚아야 하나요?"

Vector:  "도박" ≈ "계약" (의미만 보면 비슷)
         조문 번호 구분 약함 ❌

BM25:    "공서양속" 키워드 정확 매칭
         민법 제103조 찾기! ✅

Hybrid:  Vector + BM25 결합 = 완벽! ✅✅

Re-rank: 질문-문서 직접 평가
         법적 상황 이해! ✅✅✅
```

## 📋 실습 구성
1. Vector vs BM25 성능 비교
2. Hybrid Search (RRF)
3. Re-ranking (Cross-Encoder)
4. 퀴즈: 상황별 검색 방법 선택
5. 파라미터 튜닝 (alpha 조정)

In [ ]:
# 라이브러리 설치 및 import
!pip install -q langchain-community faiss-cpu sentence-transformers pandas matplotlib rank-bm25 FlagEmbedding

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import pandas as pd
import matplotlib.pyplot as plt
from rank_bm25 import BM25Okapi
import numpy as np
from FlagEmbedding import FlagReranker

print("✅ Ready!")

In [ ]:
# 법률 문서 (조문, 법률 용어, 판례)
legal_docs = [
    {
        "content": """민법 제103조 (반사회질서의 법률행위)
조문: 선량한 풍속 기타 사회질서에 위반한 사항을 내용으로 하는 법률행위는 무효로 한다.
적용: 공서양속 위반 계약, 불법 원인 급여
판례: 대법원 2015다234567 - 도박 채무 계약 무효
키워드: 공서양속, 무효, 반사회적""",
        "law": "민법 제103조",
        "code": "103",
        "category": "민법"
    },
    {
        "content": """민법 제750조 (불법행위의 내용)
조문: 고의 또는 과실로 인한 위법행위로 타인에게 손해를 가한 자는 그 손해를 배상할 책임이 있다.
적용: 교통사고, 명예훼손, 재산 침해
판례: 대법원 2018다123456 - 교통사고 손해배상
키워드: 불법행위, 손해배상, 과실""",
        "law": "민법 제750조",
        "code": "750",
        "category": "민법"
    },
    {
        "content": """형법 제257조 (상해와 존속상해)
조문: 사람의 신체를 상해한 자는 7년 이하의 징역, 10년 이하의 자격정지 또는 1천만원 이하의 벌금에 처한다.
적용: 폭행으로 인한 상해, 신체 손상
판례: 대법원 2020도56789 - 폭행 상해 사건
키워드: 상해, 폭행, 신체""",
        "law": "형법 제257조",
        "code": "257",
        "category": "형법"
    },
    {
        "content": """형법 제329조 (절도)
조문: 타인의 재물을 절취한 자는 6년 이하의 징역 또는 1천만원 이하의 벌금에 처한다.
적용: 타인 재산 무단 취득
판례: 대법원 2019도12345 - 절도 사건
키워드: 절도, 재물, 절취""",
        "law": "형법 제329조",
        "code": "329",
        "category": "형법"
    },
    {
        "content": """상법 제15조 (상호)
조문: 상인은 그 성명 기타의 명칭으로 상호를 정할 수 있다.
적용: 회사명 등록, 영업표지
판례: 대법원 2017다78901 - 상호권 침해
키워드: 상호, 상인, 명칭""",
        "law": "상법 제15조",
        "code": "15",
        "category": "상법"
    },
    
    # Confuser 문서 - 비슷한 법조문
    {
        "content": """민법 제104조 (불공정한 법률행위)
조문: 당사자의 궁박, 경솔 또는 무경험으로 인하여 현저하게 공정을 잃은 법률행위는 무효로 한다.
적용: 폭리행위, 불공정 계약
판례: 대법원 2016다345678 - 불공정 계약 무효
키워드: 불공정, 무효, 폭리""",
        "law": "민법 제104조",
        "code": "104",
        "category": "민법"
    },
    {
        "content": """민법 제751조 (재산 이외의 손해의 배상)
조문: 타인의 신체, 자유 또는 명예를 해하거나 기타 정신상 고통을 가한 자는 재산 이외의 손해에 대하여도 배상할 책임이 있다.
적용: 위자료, 정신적 손해
판례: 대법원 2019다234567 - 명예훼손 위자료
키워드: 위자료, 정신적 손해, 명예""",
        "law": "민법 제751조",
        "code": "751",
        "category": "민법"
    },
    {
        "content": """형법 제258조 (중상해와 존속중상해)
조문: 사람의 신체를 상해하여 생명에 대한 위험을 발생하게 한 자는 1년 이상 10년 이하의 징역에 처한다.
적용: 생명 위협 상해
판례: 대법원 2021도23456 - 중상해 사건
키워드: 중상해, 생명위험""",
        "law": "형법 제258조",
        "code": "258",
        "category": "형법"
    },
    {
        "content": """형법 제330조 (야간주거침입절도)
조문: 야간에 사람의 주거에 침입하여 타인의 재물을 절취한 자는 10년 이하의 징역에 처한다.
적용: 야간 주택 침입 절도
판례: 대법원 2020도67890 - 야간주거침입절도
키워드: 야간, 주거침입, 절도""",
        "law": "형법 제330조",
        "code": "330",
        "category": "형법"
    },
    {
        "content": """상법 제16조 (상호의 배타적 사용권)
조문: 상호는 상인의 영업과 함께 양도하는 경우에 한하여 양도할 수 있다.
적용: 상호권 양도
키워드: 상호, 양도""",
        "law": "상법 제16조",
        "code": "16",
        "category": "상법"
    },
    
    # 노이즈 문서
    {
        "content": """일반 법률 가이드
법률 상담 시 유의사항
- 정확한 사실관계 전달
- 증거 자료 준비""",
        "law": None,
        "category": "일반"
    }
]

print(f"⚖️  법률 문서 준비!")
print(f"   총 {len(legal_docs)}개 문서")
print()
print("📋 문서 구성:")
print("   1. 민법 제103조 - 반사회질서의 법률행위 (공서양속)")
print("   2. 민법 제750조 - 불법행위 (손해배상)")
print("   3. 형법 제257조 - 상해")
print("   4. 형법 제329조 - 절도")
print("   5. 상법 제15조 - 상호")
print("   + Confuser: 유사 조문 (제104조, 제751조, 제258조, 제330조, 제16조)")
print("   + 노이즈: 일반 법률 가이드")
print()
print("💡 법률 문서 전략:")
print("   - 조문 번호: 제103조, 제750조, 제257조 (정확한 매칭)")
print("   - 법률 용어: 공서양속, 불법행위, 상해 (전문 용어)")
print("   - 판례 번호: 2015다234567, 2018다123456 (숫자 매칭)")
print()
print("🎯 기대 동작:")
print("   - Vector: 조문 번호/법률 용어 구분 약함 ❌")
print("   - BM25: '제103조', '공서양속' 정확 매칭 ✅")
print("   - Hybrid: 의미 + 조문 결합 ✅✅")
print("   - Re-ranking: 법적 상황 이해! ✅✅✅")
print()
print("🔬 DeepMind 연구:")
print("   - 임베딩 모델: 숫자, 조문, 법률 용어 약함")
print("   - '제103조' vs '제104조' 구분 어려움")
print("   - BM25 + Re-ranking 필수!")

In [ ]:
# 벡터스토어 구축
documents = [Document(
    page_content=doc["content"], 
    metadata={k: v for k, v in doc.items() if k != "content"}
) for doc in legal_docs]

# 일반 임베딩 모델 사용 (법률 용어/조문 특화 학습 안됨)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = FAISS.from_documents(documents, embeddings)

print("✅ 벡터스토어 구축 완료!")
print(f"   임베딩 모델: paraphrase-multilingual-MiniLM-L12-v2")
print(f"   문서 수: {len(documents)}개")
print()
print("💡 일반 임베딩 모델 + 법률 도메인:")
print("   - 일반 MiniLM 모델: 법률 용어/조문 학습 안됨")
print("   - '제103조' vs '제104조' 조문 번호 구분 약함")
print("   - '공서양속' vs '불공정' 법률 용어 혼동")
print()
print("🎯 법률 도메인 특징:")
print("   - 조문 번호: 제103조, 제750조, 제257조 → 임베딩 약함")
print("   - 법률 용어: 공서양속, 불법행위, 상해 → 전문 용어 구분 어려움")
print("   - 판례 번호: 2015다234567 → 숫자 매칭 약함")
print()
print("📊 예상 차이:")
print("   - Vector 성능 ↓↓ (조문/용어 + 일반 모델)")
print("   - BM25 성능 ↑↑ (조문/용어 정확 매칭!)")
print("   - Hybrid ↑↑↑ (의미 + 정확한 매칭)")
print("   - Re-ranking ↑↑↑↑ (법적 상황 이해!)")

## 🔴 Step 1: 벡터 검색의 법률 용어 매칭 한계

### 문제 상황
**조문 번호**, **법률 용어**, **판례 번호** 같은 정확한 매칭이 필요할 때?

### 벡터 검색의 특징
- ✅ **의미 유사도** 강력: "다쳤어요" ≈ "상해" ≈ "신체 손상"
- ❌ **조문/용어 구분** 약함: "제103조" vs "제104조" vs "제750조" 구분 어려움

### 테스트 케이스 5가지
1. **법적 상황 → 조문**: "도박 빚" → 민법 제103조 (공서양속)
2. **의미 유사도**: "교통사고 배상" ≈ "불법행위 손해배상"
3. **정확한 조문**: "신체 상해" → 형법 제257조
4. **법률 용어**: "재물 절도" → 형법 제329조
5. **상호 등록**: "회사명" → 상법 제15조

지금부터 **Vector, BM25, Hybrid, Re-ranking** 순서대로 테스트합니다!

### 🔬 연구 결과
- 임베딩은 **숫자, 조문 번호, 전문 법률 용어**에서 약함
- 의료/법률같이 도메인 특화 용어가 많은 분야
- **Hybrid Search가 필수!**

In [ ]:
# ========================================
# 테스트 케이스 정의 (5개)
# ========================================

test_cases = [
    {
        "query": "공서양속 위반 계약",
        "expected_law": "민법 제103조",
        "expected_category": "민법",
        "difficulty": "어려움 (전문 법률 용어)"
    },
    {
        "query": "불법행위 손해배상",
        "expected_law": "민법 제750조",
        "expected_category": "민법",
        "difficulty": "중간 (법률 용어 매칭)"
    },
    {
        "query": "신체 상해 죄",
        "expected_law": "형법 제257조",
        "expected_category": "형법",
        "difficulty": "중간 (상해 키워드)"
    },
    {
        "query": "재물 절도",
        "expected_law": "형법 제329조",
        "expected_category": "형법",
        "difficulty": "쉬움 (절도 키워드 명확)"
    },
    {
        "query": "상호 등록",
        "expected_law": "상법 제15조",
        "expected_category": "상법",
        "difficulty": "쉬움 (상호 키워드 명확)"
    }
]

print("=" * 70)
print("📋 테스트 케이스 (5개) - 법률 상황 매칭")
print("=" * 70)
for i, test in enumerate(test_cases, 1):
    print(f"\n{i}. 질문: \"{test['query']}\"")
    print(f"   찾아야 할 조문: {test['expected_law']}")
    print(f"   난이도: {test['difficulty']}")

print("\n" + "=" * 70)
print("🎯 목표 성능: Vector 2/5 → BM25 3/5 → Hybrid 4/5 → Re-rank 5/5")
print("=" * 70)

# ========================================
# Vector Search 테스트
# ========================================
print("\n\n🔍 Vector Search 테스트 시작")
print("💡 특징: 의미 유사도로 검색")
print("=" * 70)

vector_success = 0

for i, test in enumerate(test_cases, 1):
    query = test['query']
    expected_category = test['expected_category']
    expected_law = test['expected_law']
    
    print(f"\n━━━ Test {i}/5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print(f"📝 질문: \"{query}\"")
    print(f"🎯 찾아야 할 것: {expected_category} ({expected_law})")
    
    # Vector 검색
    results = vectorstore.similarity_search(query, k=1)
    top_doc = results[0]
    
    category_match = top_doc.metadata.get('category') == expected_category
    law_match = top_doc.metadata.get('law') == expected_law
    success = category_match and law_match
    
    print(f"\n🔍 검색 결과:")
    print(f"   [{top_doc.metadata.get('category')}] {top_doc.page_content[:50]}...")
    print(f"   카테고리: {top_doc.metadata.get('category')} {'✅' if category_match else '❌'}")
    print(f"   조문: {top_doc.metadata.get('law')} {'✅' if law_match else '❌'}")
    
    if success:
        print(f"\n   ✅ 성공!")
        vector_success += 1
    else:
        print(f"\n   ❌ 실패!")

print("\n" + "=" * 70)
print(f"📊 Vector 검색 최종 결과: {vector_success}/5 ({vector_success/5*100:.0f}%)")
print("=" * 70)

## 💡 Step 2: BM25 키워드 검색 소개

### BM25란?
**"Best Matching 25"** - 정확한 단어 매칭을 위한 검색 알고리즘

### Vector vs BM25

| 항목 | Vector Search | BM25 |
|------|---------------|------|
| 검색 방식 | 의미 유사도 (임베딩) | 키워드 매칭 (단어 빈도) |
| 장점 | 비슷한 의미 찾기 | 정확한 단어 찾기 |
| 단점 | 키워드 매칭 약함 | 의미 이해 부족 |
| 법률 예시 | "상해" ≈ "중상해" | "제103조" = "제103조" (정확) |

### BM25 작동 원리 (간단히)

```
1. 문서 토크나이징
   "민법 제103조 공서양속" → ["민법", "제103조", "공서양속"]

2. 쿼리 토크나이징
   "제103조 무효" → ["제103조", "무효"]

3. 매칭 점수 계산
   - "제103조" 정확 매칭 ✅ → 점수 +10
   - "무효" 부분 매칭 ✅ → 점수 +5
   
4. 점수 높은 순 정렬
```

### 비유
- **Vector**: 느낌으로 찾기 ("이런 법적 상황")
- **BM25**: 조문집에서 찾기 ("정확히 제103조")

### 실습: BM25 구축하기

In [ ]:
# BM25 인덱스 구축 (법률 문서용 토크나이저)
import re

def legal_tokenizer(text):
    """
    법률 문서용 토크나이저
    - 조문 번호 (제103조) 특별 처리
    - 판례 번호 (2015다234567) 특별 처리
    - 숫자 보존
    """
    # 조문 번호 보존 (제103조, 제750조 등)
    text = re.sub(r'(제\d+조)', r' \1 ', text)
    
    # 판례 번호 보존 (2015다234567 등)
    text = re.sub(r'(\d{4}[다도가나]\d+)', r' \1 ', text)
    
    # 순수 숫자 보존 (103, 750, 257 등)
    text = re.sub(r'\b(\d+)\b', r' \1 ', text)
    
    # 공백으로 분리
    tokens = [t for t in text.split() if t.strip()]
    
    return tokens

# 1. 문서 내용 추출
corpus = [doc.page_content for doc in documents]

# 2. 법률 문서용 토크나이징
tokenized_corpus = [legal_tokenizer(doc) for doc in corpus]

# 3. BM25 인덱스 생성
bm25 = BM25Okapi(tokenized_corpus)

print("✅ BM25 인덱스 구축 완료! (법률 문서용 토크나이저)")
print(f"   문서 수: {len(corpus)}개")
print()
print("📋 토크나이징 예시:")
print(f"   원본: {corpus[0][:40]}...")
print(f"   토큰: {tokenized_corpus[0][:8]}...")
print()
print("💡 개선 사항:")
print("   - 조문 번호 보존: '제103조', '제750조', '제257조'")
print("   - 판례 번호 보존: '2015다234567', '2018다123456'")
print("   - 숫자 보존: '103', '750', '257'")
print("   - BM25 매칭 정확도 극대화!")

In [ ]:
# BM25 검색 함수 (법률 문서용 토크나이저 적용)
def bm25_search(query, k=3):
    """BM25로 문서 검색 (법률 토크나이저 사용)"""
    # 쿼리 토크나이징 (같은 토크나이저 사용!)
    tokenized_query = legal_tokenizer(query)
    
    # BM25 점수 계산
    scores = bm25.get_scores(tokenized_query)
    
    # 점수 높은 순으로 정렬
    top_indices = np.argsort(scores)[::-1][:k]
    
    # 결과 반환
    results = [(documents[i], scores[i]) for i in top_indices]
    return results

print("✅ BM25 검색 함수 준비 완료! (법률 문서용 토크나이저)")
print()
print("💡 쿼리 처리 예시:")
print("   '제103조 무효' → ['제103조', '무효']")
print("   '불법행위 손해배상' → ['불법행위', '손해배상']")
print("   '상해 형법' → ['상해', '형법']")

## 🔍 Step 3: BM25 검색 테스트

### 실습: 직접 BM25로 검색해보기

동일한 5가지 법률 쿼리를 **BM25로 다시** 해봅시다!
- 벡터 검색과 어떻게 다른지 비교해보세요.
- 법률 용어와 조문 번호가 정확히 매칭되는지 확인하세요!

In [ ]:
# ========================================
# BM25 Search 테스트
# ========================================

print("\n\n🔍 BM25 Search 테스트 시작")
print("💡 특징: 키워드 정확 매칭")
print("=" * 70)

bm25_success = 0

for i, test in enumerate(test_cases, 1):
    query = test['query']
    expected_category = test['expected_category']
    expected_law = test['expected_law']
    
    print(f"\n━━━ Test {i}/5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print(f"📝 질문: \"{query}\"")
    print(f"🎯 찾아야 할 것: {expected_category} ({expected_law})")
    
    # BM25 검색
    results = bm25_search(query, k=1)
    top_doc, score = results[0]
    
    category_match = top_doc.metadata.get('category') == expected_category
    law_match = top_doc.metadata.get('law') == expected_law
    success = category_match and law_match
    
    print(f"\n🔍 검색 결과 (BM25 점수: {score:.2f}):")
    print(f"   [{top_doc.metadata.get('category')}] {top_doc.page_content[:50]}...")
    print(f"   카테고리: {top_doc.metadata.get('category')} {'✅' if category_match else '❌'}")
    print(f"   조문: {top_doc.metadata.get('law')} {'✅' if law_match else '❌'}")
    
    if success:
        print(f"\n   ✅ 성공!")
        bm25_success += 1
    else:
        print(f"\n   ❌ 실패!")

print("\n" + "=" * 70)
print(f"📊 BM25 검색 최종 결과: {bm25_success}/5 ({bm25_success/5*100:.0f}%)")
print("=" * 70)

## 🚀 Step 4: Hybrid Search (Vector + BM25)

### 최강 조합!

**Vector의 장점 + BM25의 장점 = 완벽!**

```
Vector:  의미 유사도 검색 👍
BM25:    키워드 정확도 검색 👍
         ↓ 결합
Hybrid:  의미 + 키워드 둘 다! 🎯
```

### RRF (Reciprocal Rank Fusion)

**"순위 역수 융합"** - 두 검색 결과를 합치는 방법

#### 공식
```
RRF_score(doc) = Σ 1 / (k + rank)

k = 60 (상수)
rank = 문서 순위 (1, 2, 3, ...)
```

#### 예시
```
문서 A:
  Vector: 1위 → 1/(60+1) = 0.016
  BM25:   3위 → 1/(60+3) = 0.016
  합계: 0.032

문서 B:
  Vector: 2위 → 1/(60+2) = 0.016
  BM25:   1위 → 1/(60+1) = 0.016
  합계: 0.032

문서 C:
  Vector: 1위 → 1/(60+1) = 0.016
  BM25:   1위 → 1/(60+1) = 0.016
  합계: 0.032 (가장 높음!) ✅
```

### 핵심 아이디어
- 두 검색에서 **모두 상위권**인 문서가 최고 점수!
- 한쪽만 좋으면 점수 낮음

### 구현해봅시다!

In [ ]:
# Hybrid Search 함수 (RRF)
def hybrid_search(query, k=3, alpha=0.3):
    """
    Hybrid Search using Reciprocal Rank Fusion (RRF)
    
    Args:
        query: 검색 쿼리
        k: 반환할 문서 수
        alpha: Vector 가중치 (0~1, BM25 가중치는 1-alpha)
              기본값 0.3 = Vector 30%, BM25 70%
    """
    # 1. Vector 검색
    vector_results = vectorstore.similarity_search(query, k=10)
    
    # 2. BM25 검색
    bm25_results = bm25_search(query, k=10)
    
    # 3. RRF 점수 계산
    rrf_scores = {}
    k_constant = 60  # RRF 상수
    
    # Vector 검색 결과 점수화
    for rank, doc in enumerate(vector_results, 1):
        doc_id = doc.page_content  # 문서 식별자
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + alpha * (1 / (k_constant + rank))
    
    # BM25 검색 결과 점수화
    for rank, (doc, score) in enumerate(bm25_results, 1):
        doc_id = doc.page_content
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + (1 - alpha) * (1 / (k_constant + rank))
    
    # 4. 점수 높은 순으로 정렬
    sorted_docs = sorted(rrf_scores.items(), key=lambda x: -x[1])
    
    # 5. 상위 k개 문서 반환
    results = []
    for doc_content, score in sorted_docs[:k]:
        # 원본 Document 객체 찾기
        doc = next(d for d in documents if d.page_content == doc_content)
        results.append((doc, score))
    
    return results

print("✅ Hybrid Search 함수 준비 완료!")
print()
print("💡 RRF 공식:")
print("   RRF_score(doc) = α × Vector_RRF + (1-α) × BM25_RRF")
print("   RRF(rank) = 1 / (60 + rank)")
print()
print("💡 가중치 조정 (중요!):")
print("   alpha = 0.3 (Vector 30%, BM25 70%)")
print("   이유: Vector가 약할 때 BM25 가중치를 높여서 보완!")
print()
print("🎯 가중치 전략:")
print("   - 강한 Vector 모델: alpha=0.5 (균형)")
print("   - 약한 Vector 모델: alpha=0.3 (BM25 중시) ← 현재")
print("   - 도메인 특화: alpha=0.2~0.3 (정확한 키워드 중요)")
print()
print("📊 Alpha 효과:")
print("   - alpha ↑ (0.5~0.7): Vector 영향 증가")
print("   - alpha ↓ (0.2~0.4): BM25 영향 증가")

In [ ]:
# ========================================
# Hybrid Search 테스트
# ========================================

print("\n\n🔍 Hybrid Search 테스트 시작")
print("💡 특징: Vector + BM25 RRF 결합")
print("=" * 70)

hybrid_success = 0

for i, test in enumerate(test_cases, 1):
    query = test['query']
    expected_category = test['expected_category']
    expected_law = test['expected_law']
    
    print(f"\n━━━ Test {i}/5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print(f"📝 질문: \"{query}\"")
    print(f"🎯 찾아야 할 것: {expected_category} ({expected_law})")
    
    # Hybrid 검색
    results = hybrid_search(query, k=1)
    top_doc, score = results[0]
    
    category_match = top_doc.metadata.get('category') == expected_category
    law_match = top_doc.metadata.get('law') == expected_law
    success = category_match and law_match
    
    print(f"\n🔍 검색 결과 (Hybrid RRF: {score:.4f}):")
    print(f"   [{top_doc.metadata.get('category')}] {top_doc.page_content[:50]}...")
    print(f"   카테고리: {top_doc.metadata.get('category')} {'✅' if category_match else '❌'}")
    print(f"   조문: {top_doc.metadata.get('law')} {'✅' if law_match else '❌'}")
    
    if success:
        print(f"\n   ✅ 성공!")
        hybrid_success += 1
    else:
        print(f"\n   ❌ 실패!")

print("\n" + "=" * 70)
print(f"📊 Hybrid 검색 최종 결과: {hybrid_success}/5 ({hybrid_success/5*100:.0f}%)")
print("=" * 70)

In [ ]:
# Re-ranking 함수 (BAAI/bge-reranker-v2-m3)
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

def rerank_search(query, k=3, top_k_candidates=10):
    """
    Re-ranking using Cross-Encoder (BAAI/bge-reranker-v2-m3)
    
    Args:
        query: 검색 쿼리
        k: 반환할 문서 수
        top_k_candidates: Hybrid에서 가져올 후보 문서 수
    """
    # 1. Hybrid Search로 후보 문서 검색
    hybrid_results = hybrid_search(query, k=top_k_candidates)
    
    # 2. Cross-Encoder로 재점수화
    pairs = [[query, doc.page_content] for doc, _ in hybrid_results]
    scores = reranker.compute_score(pairs, normalize=True)
    
    # 3. 점수 높은 순으로 재정렬
    if isinstance(scores, float):
        scores = [scores]
    
    reranked = sorted(zip(hybrid_results, scores), key=lambda x: -x[1])
    
    # 4. 상위 k개 반환
    results = [(doc, score) for (doc, _), score in reranked[:k]]
    return results

print("✅ Re-ranking 함수 준비 완료!")
print()
print("💡 Re-ranking (Cross-Encoder):")
print("   모델: BAAI/bge-reranker-v2-m3")
print("   기능: 질문-문서를 직접 함께 분석해 관련성 평가")
print()
print("🎯 장점:")
print("   - 질문과 문서를 동시에 인코딩 (문맥 이해!)")
print("   - Vector/BM25보다 훨씬 정확한 관련성 평가")
print("   - 최종 정확도 극대화!")

## 📊 Step 5: Re-ranking (최종 단계!)

### Cross-Encoder로 법적 상황 완벽 이해

**Re-ranking이란?**
- Hybrid Search 결과를 **Cross-Encoder**로 재평가
- 질문-문서를 **함께 인코딩**해서 직접 관련성 점수 계산
- 가장 정확하지만 느림 (Top-100 → Top-10 선택 전략)

### Vector vs BM25 vs Hybrid vs Re-ranking

```
Vector:  "도박" ≈ "계약" (의미 비슷)
BM25:    "공서양속" = "공서양속" (정확 매칭)
Hybrid:  Vector + BM25 결합
Re-rank: ["도박 빚", "민법 제103조 공서양속"] → 직접 관련성 0.95! ✅
```

### 법률 예시

**질문**: "공서양속 위반 계약"

- **Vector**: 유사한 여러 민법 조문 혼동
- **BM25**: "공서양속" 키워드로 제103조 찾음
- **Hybrid**: 의미 + 키워드 결합
- **Re-rank**: 질문의 법적 의미 완벽 이해 → 민법 제103조!

### 실습: Re-ranking 테스트

In [ ]:
# ========================================
# Re-ranking 테스트 (최종 단계!)
# ========================================

print("\n\n🔍 Re-ranking 테스트 시작 (최종 단계!)")
print("💡 특징: Cross-Encoder 문맥 이해!")
print("=" * 70)

rerank_success = 0

for i, test in enumerate(test_cases, 1):
    query = test['query']
    expected_category = test['expected_category']
    expected_law = test['expected_law']
    
    print(f"\n━━━ Test {i}/5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print(f"📝 질문: \"{query}\"")
    print(f"🎯 찾아야 할 것: {expected_category} ({expected_law})")
    
    # Re-ranking 검색
    results = rerank_search(query, k=1)
    top_doc, score = results[0]
    
    category_match = top_doc.metadata.get('category') == expected_category
    law_match = top_doc.metadata.get('law') == expected_law
    success = category_match and law_match
    
    print(f"\n🔍 검색 결과 (Re-rank: {score:.4f}):")
    print(f"   [{top_doc.metadata.get('category')}] {top_doc.page_content[:50]}...")
    print(f"   카테고리: {top_doc.metadata.get('category')} {'✅' if category_match else '❌'}")
    print(f"   조문: {top_doc.metadata.get('law')} {'✅' if law_match else '❌'}")
    
    if success:
        print(f"\n   ✅ 성공!")
        rerank_success += 1
    else:
        print(f"\n   ❌ 실패!")

print("\n" + "=" * 70)
print(f"📊 Re-ranking 최종 결과: {rerank_success}/5 ({rerank_success/5*100:.0f}%)")
print("=" * 70)

print(f"\n🚀 성능 progression:")
print(f"   Vector:  {vector_success}/5 ({vector_success/5*100:.0f}%)")
print(f"   BM25:    {bm25_success}/5 ({bm25_success/5*100:.0f}%)")
print(f"   Hybrid:  {hybrid_success}/5 ({hybrid_success/5*100:.0f}%)")
print(f"   Re-rank: {rerank_success}/5 ({rerank_success/5*100:.0f}%) ✨")

---

## 🎯 퀴즈: 상황별 검색 방법 선택

다음 상황에 가장 적합한 검색 방법은?

**상황**: "민법 제103조를 정확히 찾고 싶어요"

**A)** Vector Search (의미 유사도)  
**B)** BM25 (키워드 정확 매칭)  
**C)** Hybrid (Vector + BM25)  
**D)** Re-ranking (Cross-Encoder)

아래 셀에서 각각 테스트해보세요!

In [ ]:
# 4가지 방법으로 "민법 제103조" 검색 테스트

test_query = "민법 제103조"
expected = "민법 제103조"

print("🔍 4가지 방법 비교")
print("=" * 60)
print(f"질문: {test_query}")
print(f"찾아야 할 것: {expected}")
print()

# A) Vector
print("A) Vector Search:")
v_results = vectorstore.similarity_search(test_query, k=1)
v_law = v_results[0].metadata.get('law')
print(f"   결과: {v_law} {'✅' if v_law == expected else '❌'}")
print()

# B) BM25
print("B) BM25:")
b_results = bm25_search(test_query, k=1)
b_law = b_results[0][0].metadata.get('law')
print(f"   결과: {b_law} {'✅' if b_law == expected else '❌'}")
print()

# C) Hybrid
print("C) Hybrid:")
h_results = hybrid_search(test_query, k=1)
h_law = h_results[0][0].metadata.get('law')
print(f"   결과: {h_law} {'✅' if h_law == expected else '❌'}")
print()

# D) Re-ranking
print("D) Re-ranking:")
r_results = rerank_search(test_query, k=1)
r_law = r_results[0][0].metadata.get('law')
print(f"   결과: {r_law} {'✅' if r_law == expected else '❌'}")
print()

print("=" * 60)
print("💡 정답: B, C, D 모두 가능!")
print("   조문 번호는 정확한 키워드 매칭이 중요!")

<details>
<summary>📝 해설 보기 (클릭)</summary>

**정답: B, C, D 모두 가능!**

**A) Vector Search** ❌
- "제103조" vs "제104조" 구분 어려움
- 숫자는 의미가 없어서 임베딩이 약함

**B) BM25** ✅
- "제103조" 키워드 정확 매칭!
- 조문 번호는 BM25가 강함

**C) Hybrid** ✅
- BM25로 정확한 조문 찾음
- Vector의 약점을 BM25가 보완

**D) Re-ranking** ✅
- 가장 정확하지만 느림
- 질문-문서 직접 관련성 평가

**핵심:**
- 조문 번호, 판례 번호 → BM25 필수!
- 법적 상황 이해 → Re-ranking 최고!

</details>

---

## 🎛️ 실습: 파라미터 튜닝 (alpha 조정)

### alpha란?

Hybrid Search에서 **Vector와 BM25의 가중치**를 조절하는 파라미터

```python
alpha = 0.3  # Vector 30%, BM25 70%
alpha = 0.5  # Vector 50%, BM25 50% (균형)
alpha = 0.7  # Vector 70%, BM25 30%
```

### alpha 값에 따른 효과

- **alpha ↑ (0.7~0.9)**: Vector 영향 증가 → 의미 이해 중시
- **alpha ↓ (0.1~0.3)**: BM25 영향 증가 → 키워드 정확도 중시

### 실습: alpha 값을 바꿔가며 성능 확인!

현재 법률 문서에서 최적의 alpha는?

In [ ]:
# 여기를 수정하세요! 👇
# TODO: alpha 값을 변경하면서 성능이 어떻게 바뀌는지 확인하세요

alpha_value = 0.5  # 0.1 ~ 0.9 사이 값으로 변경해보세요

print(f"🎛️ Alpha = {alpha_value} 테스트")
print(f"   Vector: {alpha_value*100:.0f}%, BM25: {(1-alpha_value)*100:.0f}%")
print("=" * 60)
print()

# Hybrid 검색 함수 (alpha 파라미터 적용)
def hybrid_search_with_alpha(query, k=3, alpha=0.3):
    vector_results = vectorstore.similarity_search(query, k=10)
    bm25_results = bm25_search(query, k=10)
    
    rrf_scores = {}
    k_constant = 60
    
    for rank, doc in enumerate(vector_results, 1):
        doc_id = doc.page_content
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + alpha * (1 / (k_constant + rank))
    
    for rank, (doc, score) in enumerate(bm25_results, 1):
        doc_id = doc.page_content
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + (1 - alpha) * (1 / (k_constant + rank))
    
    sorted_docs = sorted(rrf_scores.items(), key=lambda x: -x[1])
    results = []
    for doc_content, score in sorted_docs[:k]:
        doc = next(d for d in documents if d.page_content == doc_content)
        results.append((doc, score))
    
    return results

# 5개 테스트 케이스로 성능 측정
success_count = 0

for i, test in enumerate(test_cases, 1):
    query = test['query']
    expected_law = test['expected_law']
    
    results = hybrid_search_with_alpha(query, k=1, alpha=alpha_value)
    found_law = results[0][0].metadata.get('law')
    
    success = (found_law == expected_law)
    if success:
        success_count += 1
    
    print(f"{i}. {query[:15]}... → {found_law} {'✅' if success else '❌'}")

print()
print("=" * 60)
print(f"📊 성공률: {success_count}/5 ({success_count/5*100:.0f}%)")
print()
print("💡 실험해보세요:")
print("   alpha=0.1: BM25 90% → 조문 번호 매칭 극대화")
print("   alpha=0.3: BM25 70% → 법률 문서에 적합 (현재 기본값)")
print("   alpha=0.5: 균형 50:50")
print("   alpha=0.7: Vector 70% → 의미 이해 중시")

<details>
<summary>📝 해설 보기 (클릭)</summary>

**최적 alpha 값: 0.2 ~ 0.3**

**이유:**
- 법률 문서는 **조문 번호, 법률 용어**가 중요
- Vector는 "제103조" vs "제104조" 구분 약함
- BM25가 더 중요 → alpha를 낮춰서 BM25 비중 증가!

**실험 결과 (예상):**
```
alpha=0.1: 5/5 (BM25 90%) ← 최고!
alpha=0.3: 4/5 (BM25 70%) ← 기본값
alpha=0.5: 3/5 (균형)
alpha=0.7: 2/5 (Vector 70%) ← 낮음
```

**도메인별 최적 alpha:**
- **법률 문서**: 0.2~0.3 (조문 번호 중요)
- **일반 Q&A**: 0.5 (균형)
- **창의적 검색**: 0.6~0.7 (의미 중시)

**핵심:**
도메인 특성에 맞춰 alpha 조정이 중요!

</details>

In [ ]:
# Performance Comparison
methods = ['Vector', 'BM25', 'Hybrid', 'Re-ranking']
success_counts = [vector_success, bm25_success, hybrid_success, rerank_success]
colors = ['lightblue', 'lightgreen', 'gold', 'coral']

plt.figure(figsize=(12, 6))
bars = plt.bar(methods, success_counts, color=colors, alpha=0.8, edgecolor='black', linewidth=2)
plt.ylabel('Success Count (out of 5)', fontsize=12, fontweight='bold')
plt.title('Search Performance: Vector → BM25 → Hybrid → Re-ranking', fontsize=14, fontweight='bold')
plt.ylim(0, 6)
plt.grid(axis='y', alpha=0.3, linestyle='--')

for bar, count in zip(bars, success_counts):
    height = bar.get_height()
    label = f'{count}/5'
    if count >= 4:
        label += ' ✅'
    elif count <= 2:
        label += ' ❌'
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.1, 
             label, ha='center', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print()
print(f"Performance Summary:")
print(f"  Vector:     {vector_success}/5 ({vector_success/5*100:.0f}%)")
print(f"  BM25:       {bm25_success}/5 ({bm25_success/5*100:.0f}%)")
print(f"  Hybrid:     {hybrid_success}/5 ({hybrid_success/5*100:.0f}%)")
print(f"  Re-ranking: {rerank_success}/5 ({rerank_success/5*100:.0f}%)")
print()
print("Key Insights:")
print("  • Vector: Weak on legal article numbers (조문 번호 약함)")
print("  • BM25: Good at exact legal term matching (법률 용어 정확)")
print("  • Hybrid: Combines meaning + exact matching! (의미+정확도)")
if rerank_success >= 4:
    print(f"  • Re-ranking: Excellent! Legal situations understood! (법적 상황 완벽 이해!)")
elif rerank_success > hybrid_success:
    print(f"  • Re-ranking: Best! (+{rerank_success - hybrid_success} over Hybrid)")
else:
    print("  • Re-ranking: Legal context understanding")

## 🎓 핵심 정리

### Hybrid Search + Re-ranking이란?

**1단계: Hybrid Search (빠른 후보 검색)**
```
Vector: 의미 이해
BM25: 키워드 정확 매칭
↓ RRF 결합
Hybrid: Top-100 후보 검색
```

**2단계: Re-ranking (정확한 최종 선택)**
```
Cross-Encoder: 질문-문서 직접 평가
↓ 점수 높은 순 재정렬
Final: Top-10 최종 결과
```

### 언제 사용?
- ✅ 법률 문서 (조문 번호, 판례)
- ✅ 의료 문서 (질병명, ICD 코드)
- ✅ 기술 문서 (API, 버전, 에러 코드)

### 핵심 파라미터
- **alpha**: Vector vs BM25 가중치
  - 법률: 0.2~0.3 (BM25 중시)
  - 일반: 0.5 (균형)
  - 창의: 0.6~0.7 (Vector 중시)

---

**🚀 다음 단계:** 07번 Modular RAG

In [ ]:
# 🎉 06번 완료!

print("✅ 배운 내용:")
print("   1. Vector 검색의 조문 번호 매칭 한계")
print("   2. BM25로 정확한 키워드 매칭")
print("   3. Hybrid Search (RRF, alpha 조정)")
print("   4. Re-ranking (Cross-Encoder)")
print()
print("📊 성능 비교:")
print(f"   Vector → BM25 → Hybrid → Re-rank")
print(f"   약함 → 강함 → 더 강함 → 최강!")
print()
print("🔑 실무 전략:")
print("   1단계: Hybrid (빠름) → Top-100 후보")
print("   2단계: Re-ranking (정확) → Top-10 최종")
print()
print("🚀 다음: 07번 Modular RAG")